# 13. Interfaces, Protocols and ABCs

> Program to an interface, not an implementation
>
> Gamma, Helm, Johnson, Vlissides, First Principle of Object-Oriented Design

## Two Kinds of Protocols

The word _protocol_ has different meanings in computer science depending on context. A network protocol such as HTTP specifies commands that a client can send to a server, such as `GET`, `PUT`, and `HEAD`.

Implementing a full protocol may require several methods, but often it is OK to implement only part of it.

In [2]:
class Vowels:
    def __getitem__(self, i):
        return 'AEIOU'[i]

In [4]:
v = Vowels()
v[0]

'A'

In [5]:
v[1]

'E'

In [6]:
for c in v: print(c)

A
E
I
O
U


In [7]:
'E' in v

True

In [8]:
'Z' in v

False

## Programming Ducks

### Python Digs Sequences

The philosophy of the Python Data Model is to cooperate with essential dynamic protocols as much as possible. When it comes to sequences, Python tries hard to work with even the simplest implementations.

In [9]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank,suit) for suit in self.suits
                                                   for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

### Monkey Patching: Implementing a Protocol at Runtime

Monkey patching is dynamically changing a module, class, or function at runtime, to add features or fix bugs. For example, the gevent networking library monkey patches parts of Python’s standard library to allow lightweight concurrency without threads or `async`/`await`.

In [11]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[0, 3, 1, 6, 9, 8, 4, 2, 5, 7]

However, if we try to shuffle a FrenchDeck instance, we get an exception, as in following example

In [13]:
from random import shuffle
# from frenchdeck import FrenchDeck

deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

The error message is clear: `'FrenchDeck' object does not support item assign ment`. The problem is that `shuffle` operates _in place_, by swapping items inside the collection, and `FrenchDeck` only implements the _immutable_ sequence protocol. Mutable sequences must also provide a `__setitem__` method.

In [14]:
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='7', suit='spades'),
 Card(rank='J', suit='clubs'),
 Card(rank='K', suit='spades'),
 Card(rank='6', suit='spades'),
 Card(rank='2', suit='hearts')]

The trick is that `set_card` knows that the deck object has an attribute named `_cards`, and _cards must be a mutable sequence. The `set_card` function is then attached to the `FrenchDeck` class as the `__setitem__` special method. This is an example of _monkey patching_: changing a class or module at runtime, without touching the source code. Monkey patching is powerful, but the code that does the actual patching is very tightly coupled with the program to be patched, often handling private and undocu‐ mented attributes.

## Defensive Programming and "Fail Fast"

Defensive programming is like defensive driving: a set of practices to enhance safety even when faced with careless programmers—or drivers.

Many bugs cannot be caught except at runtime—even in mainstream statically typed languages. In a dynamically typed language, “fail fast” is excellent advice for safer and easier-to-maintain programs. Failing fast means raising runtime errors as soon as possible, for example, rejecting invalid arguments right a the beginning of a function body.

If you are afraid to get an infinite generator—not a common issue—you can begin by calling `len()` on the argument. This would reject iterators, while safely dealing with tuples, arrays, and other existing or future classes that fully implement the `Sequence` interface. Calling `len()` is usually very cheap, and an invalid argument will raise an error immediately.

On the other hand, if any iterable is acceptable, then call `iter(x)` as soon as possible to obtain an iterator. Again, if `x` is not iterable, this will fail fast with an easy-to-debug exception.

## Goose Typing

Python doesn’t have an `interface` keyword. We use abstract base classes (ABCs) to define interfaces for explicit type checking at runtime—also supported by static type checkers.

To summarize, _goose typing_ entails:
+ Subclassing from ABCs to make it explicit that you are implementing a previously defined interface.
+ Runtime type checking using ABCs instead of concrete classes as the second argument for isinstance and issubclass.

### Subclassing an ABC

Following Martelli’s advice, we’ll leverage an existing ABC, `collections.MutableSequence`, before daring to invent our own.

In [15]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value

    def __delitem__(self, position):
        del self._cards[position]

    def insert(self, position, value):
        self._cards.insert(position, value)

To write `FrenchDeck2` as a subclass of `MutableSequence`, I had to pay the price of implementing `__delitem__` and `insert`, which my examples did not require. In return, `FrenchDeck2` inherits five concrete methods from Sequence: `__contains__`, `__iter__`, `__reversed__`, `index`, and `count`. From `MutableSequence`, it gets another six methods: `append`, `reverse`, `extend`, `pop`, `remove`, and `__iadd__`—which supports the `+=` operator for in place concatenation.

### ABCs in the Standard Library

Since Python 2.6, the standard library provides several ABCs. Most are defined in the `collections.abc` module, but there are others. You can find ABCs in the `io` and `numbers` packages, for example. But the most widely used are in `collections.abc`.

In [16]:
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())
    
    def inspect(self):
        """Return a sorted tuple with the items currently inside.""" 
        items = []
        while True:
            try: 
                items.append(self.pick())
            except LookupError: 
                break
        self.load(items) 
        return tuple(items)

In [17]:
# from tombola import Tombola
class Fake(Tombola):
    def pick(self):
        return 13
    
Fake

__main__.Fake

In [18]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

## ABC Syntax Details

The standard way to declare an ABC is to subclass `abc.ABC` or any other ABC.

### Subclassing an ABC

Given the Tombola ABC, we’ll now develop two concrete subclasses that satisfy its interface.

In [19]:
import random

class BingoCage(Tombola):

    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
        
    def __call__(self):
        self.pick()

In [21]:
import random

class LottoBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)
    
    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(self._balls)

### A Virtual Subclass of an ABC

An essential characteristic of goose typing—and one reason why it deserves a waterfowl name—is the ability to register a class as a _virtual subclass_ of an ABC, even if it does not inherit from it. When doing so, we promise that the class faithfully implements the interface defined in the ABC—and Python will believe us without checking. If we lie, we’ll be caught by the usual runtime exceptions.


This is done by calling a `register` class method on the ABC. The registered class then becomes a virtual subclass of the ABC, and will be recognized as such by `issubclass`, but it does not inherit any methods or attributes from the ABC.

In [22]:
from random import randrange
# from tombola import Tombola

@Tombola.register
class TomboList(list):

    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
        
    load = list.extend

    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(self)
    
# Tombola.register(TomboList)

In [23]:
issubclass(TomboList, Tombola)

True

In [24]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

### Usage of register in Practice

Subclassing an ABC or registering with an ABC are both explicit ways of making our classes pass `issubclass` checks—as well as `isinstance` checks, which also rely on `issubclass`. But some ABCs support structural typing as well. The next section explains.

### Structural Typing with ABCs

ABCs are mostly used with nominal typing. When a class `Sub` explicitly inherits from `AnABC`, or is registered with `AnABC`, the name of `AnABC` is linked to the `Sub` class—and that’s how at runtime, `issubclass(AnABC, Sub)` returns `True`.

In contrast, structural typing is about looking at the structure of an object’s public interface to determine its type: an object is _consistent-with_ a type if it implements the methods defined in the type. Dynamic and static duck typing are two approaches to structural typing.

In [25]:
class Struggle:
    def __len__(self): return 23

from collections import abc

isinstance(Struggle(), abc.Sized)

True

In [26]:
issubclass(Struggle, abc.Sized)

True

## Static Protocols

### The Typed double Function



In [28]:
from typing import SupportsComplex
import numpy as np

c64 = np.complex64(3+4j)
isinstance(c64, complex)

False

In [29]:
isinstance(c64, SupportsComplex)

True

In [30]:
c = complex(c64)
c

(3+4j)

In [31]:
isinstance(c, SupportsComplex)

True

In [32]:
complex(c)

(3+4j)

In [33]:
isinstance(c, (complex, SupportsComplex))

True

In [34]:
import numbers
isinstance(c, numbers.Complex)

True

In [35]:
isinstance(c64, numbers.Complex)

True

### Supporting a Static Protocol